In [2]:
import pandas as pd
import numpy as np
import zipfile

!pip install nltk
import re
import nltk 
import string
from markdown import Markdown
from io import StringIO

# !pip install vaderSentiment

from openai import OpenAI


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
client = OpenAI(api_key="...")

In [4]:
with zipfile.ZipFile("../data/reddit_scrape.csv.zip") as z:
    with z.open("reddit_scrape.csv") as f:
        reddit_df = pd.read_csv(f)

reddit_df['Date'] = pd.to_datetime(reddit_df['Date'])    
reddit_df.head()

scrape_cols = [x for x in reddit_df.columns if 'craped' in x]
for col in scrape_cols: # converts strings to lists
    reddit_df[col] = reddit_df[col].apply(lambda x: eval(x) if type(x)==str else x)

    
reddit_df['game_thread_comments'] = [lst[1:] for lst in reddit_df.loc[:,'Game thread scraped']]
reddit_df['post_game_thread_comments'] = [lst[1:] if type(lst)==list else lst for lst in reddit_df.loc[:,'Post game thread Scraped']]
reddit_df['all comments'] =  reddit_df['game_thread_comments'] + reddit_df['post_game_thread_comments']
reddit_df.loc[reddit_df['all comments'].isna(),'all comments'] = \
        [lst[1:] for lst in reddit_df.loc[reddit_df['all comments'].isna(),'Game thread scraped']]
reddit_df.drop(columns=['game_thread_comments','post_game_thread_comments'],inplace=True)


/opt/homebrew/Caskroom/miniforge/base/envs/sklearn-env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


# Cleaning Text

In [5]:
def unmark_element(element, stream=None):
    if stream is None:
        stream = StringIO()
    if element.text:
        stream.write(element.text)
    for sub in element:
        unmark_element(sub, stream)
    if element.tail:
        stream.write(element.tail)
    return stream.getvalue()

# patching Markdown
Markdown.output_formats["plain"] = unmark_element
__md = Markdown(output_format="plain")
__md.stripTopLevelTags = False


def unmark(text):
    return __md.convert(text).replace(" \n"," ")

text = reddit_df['Game thread scraped'][0][1][0]
text = unmark(text).replace(" \n"," ")
text

'Dear Falcons fans: are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl? I feel your pain. Sincerely, Seahawks fans everywhere.'

In [6]:
text = text.translate(str.maketrans('', '', string.punctuation))
text

'Dear Falcons fans are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl I feel your pain Sincerely Seahawks fans everywhere'

In [7]:
def convert_super_bowl(text):
    # Use regular expression to find and replace "super bowl" with "superbowl" (case-insensitive)
    modified_text = re.sub(r'\b(super)\s+(bowl)\b', r'\1\2', text, flags=re.IGNORECASE)
    return modified_text

# Example usage:
original_text = "The Super Bowl is a great event."
converted_text = convert_super_bowl(original_text)
print(converted_text)


The SuperBowl is a great event.


In [8]:
def clean_text(sentence):
    text = unmark(sentence).replace(" \n"," ")
    text = convert_super_bowl(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Using ChatGPT

In [9]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
) 

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(3))
def sentiment_analysis(transcription):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following comment. This comment was posted on the r/nfl subreddit after a regular season NFL match. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Use one word to indicate whether the sentiment is generally positive, negative, or neutral. Be careful about sarcastic comments, indicating neutral if you are unsure about the sentiment in a comment."
            },
            {
                "role": "user",
                "content": transcription
            }
        ],
        max_tokens=1
    )
    if response.choices[0].message.content == "Positive":
        return 1
    elif response.choices[0].message.content == "Negative":
        return -1
    else:
        return 0

def list_analyser(lst):
    return [sentiment_analysis(i[0]) for i in lst]
    
def prop_calculator(analysed_lst):
    proportion_pos = analysed_lst.count(1) / len(analysed_lst)
    proportion_neg = analysed_lst.count(-1) / len(analysed_lst)
    return (proportion_pos, proportion_neg)


In [ ]:
# alpha = list_analyser(reddit_df['all comments'][0])
# alpha[:5]

In [29]:
reddit_df.columns

Index(['Season', 'Week', 'Day', 'Date', 'Time (ET)', 'Unnamed: 5', 'Favorite',
       'Score', 'Spread', 'Unnamed: 9', 'Underdog', 'Over/Under', 'Notes',
       'GameHub/RedZone', 'Game thread link?', 'Post-game thread link?',
       'Game thread', 'Post game thread', 'GameHub Scraped',
       'Game thread scraped', 'Post game thread Scraped',
       'Subscribers (in thousands)', 'all comments'],
      dtype='object')

In [21]:
transcription = clean_text(reddit_df['all comments'][0][0][0])
response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following comment. This comment was posted on the r/nfl subreddit after a regular season NFL match. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Use one word to indicate whether the sentiment is generally positive, negative, or neutral. Be careful about sarcastic comments, indicating neutral if you are unsure about the sentiment in a comment."
            },
            {
                "role": "user",
                "content": transcription
            }
        ],
        max_tokens=1
    )
response

ChatCompletion(id='chatcmpl-97Vl5AgFwKuE3U6iygSqyclYhzJEm', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Negative', role='assistant', function_call=None, tool_calls=None))], created=1711577403, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3bc1b5746c', usage=CompletionUsage(completion_tokens=1, prompt_tokens=147, total_tokens=148))

In [ ]:
problematic_rows = []
props = []
for index, row in reddit_df.iterrows():
    if index % 50 == 0:
        print(f"Scraped {index}")
        print(f"Last 5 problematic rows: {problematic_rows[-5:]}")
    try:
        props.append(prop_calculator(list_analyser(row['all comments'])))
    except:
        problematic_rows.append(index)
        props.append(np.NaN)
    pd.DataFrame({'props': props}).to_csv('progress.csv')
print(f"Scraped {len(reddit_df)}; Finished")
print(f"Last 5 problematic rows: {problematic_rows[-5:]}")

Scraped 0
Last 5 problematic rows: []


In [1]:
props

NameError: name 'props' is not defined

In [ ]:
problematic_rows